In [1]:
#import needed libraries
import pandas as pd
import json
import requests
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode
import time

In [15]:
# read training data for restaurants that were open in 2013 in AZ and were not matched with Yelp API
restaurants_old_notmerged = pd.read_pickle('./yelp_dataset_processed/yelp_df_notmerged_id.pkl')
restaurants_old_notmerged.head()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,found,google_term,google_id
2,gA5CuBxF-0CnOpGnryWJdQ,"[Mexican, Restaurants]",Phoenix,"519 W Thomas Rd\nPhoenix, AZ 85013",33.480105,-112.081361,La Paloma Mexican Food,[],True,5,4.0,AZ,business,0,La Paloma Mexican Food 519 W Thomas Rd Phoenix...,la-paloma-mexican-food-phoenix
7,sygb2W4NRadVXiWhW8bnFQ,"[Mexican, Restaurants]",Gila Bend,"616 E Pima Street\nGila Bend, AZ 85337",32.948295,-112.711322,Sofia's Mexican Food,[],True,10,3.5,AZ,business,0,Sofia's Mexican Food 616 E Pima Street Gila Be...,sofias-mexican-food-gila-bend
10,MG1IbQUurcXn8NQgXGM34Q,"[Delis, Restaurants]",Mesa,"4410 E University Dr\nMesa, AZ 85205",33.423805,-111.735789,Yee-haws,[],True,3,3.5,AZ,business,0,"Yee-haws 4410 E University Dr Mesa, AZ 85205",yee-haws-mesa
11,KqYY2rwD8ymjU9mVJwiuRQ,"[Chinese, Restaurants]",Phoenix,"9020 N 19th Ave\nPhoenix, AZ 85021",33.568297,-112.099840,Panda Gourmet,[],True,3,3.5,AZ,business,0,"Panda Gourmet 9020 N 19th Ave Phoenix, AZ 85021",panda-gourmet-phoenix
12,jvvh4Q00Hq2XyIcfmAAT2A,"[Burgers, Restaurants]",Scottsdale,"7014-590 E Camelback Rd\nScottsdale, AZ 85251",33.503323,-111.928457,Five Guys Burger and Fries,[],True,52,3.5,AZ,business,0,Five Guys Burger and Fries 7014-590 E Camelbac...,five-guys-scottsdale-2


In [24]:
restaurants_old_notmerged.reset_index(drop=True,inplace=True)

In [25]:
restaurants_old_notmerged.head()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,found,google_term,google_id
0,gA5CuBxF-0CnOpGnryWJdQ,"[Mexican, Restaurants]",Phoenix,"519 W Thomas Rd\nPhoenix, AZ 85013",33.480105,-112.081361,La Paloma Mexican Food,[],True,5,4.0,AZ,business,0,La Paloma Mexican Food 519 W Thomas Rd Phoenix...,la-paloma-mexican-food-phoenix
1,sygb2W4NRadVXiWhW8bnFQ,"[Mexican, Restaurants]",Gila Bend,"616 E Pima Street\nGila Bend, AZ 85337",32.948295,-112.711322,Sofia's Mexican Food,[],True,10,3.5,AZ,business,0,Sofia's Mexican Food 616 E Pima Street Gila Be...,sofias-mexican-food-gila-bend
2,MG1IbQUurcXn8NQgXGM34Q,"[Delis, Restaurants]",Mesa,"4410 E University Dr\nMesa, AZ 85205",33.423805,-111.735789,Yee-haws,[],True,3,3.5,AZ,business,0,"Yee-haws 4410 E University Dr Mesa, AZ 85205",yee-haws-mesa
3,KqYY2rwD8ymjU9mVJwiuRQ,"[Chinese, Restaurants]",Phoenix,"9020 N 19th Ave\nPhoenix, AZ 85021",33.568297,-112.099840,Panda Gourmet,[],True,3,3.5,AZ,business,0,"Panda Gourmet 9020 N 19th Ave Phoenix, AZ 85021",panda-gourmet-phoenix
4,jvvh4Q00Hq2XyIcfmAAT2A,"[Burgers, Restaurants]",Scottsdale,"7014-590 E Camelback Rd\nScottsdale, AZ 85251",33.503323,-111.928457,Five Guys Burger and Fries,[],True,52,3.5,AZ,business,0,Five Guys Burger and Fries 7014-590 E Camelbac...,five-guys-scottsdale-2


In [76]:
# define functions to call yelp API
def request(host, path, business_id, bearer_token, url_params=None):
    """Given a bearer token, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        bearer_token (str): OAuth bearer token, obtained using client_id and client_secret.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}{2}'.format(host, quote(path.encode('utf8')), quote(business_id.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % bearer_token,
    }

    #print(u'Querying {0} ...'.format(url))
    
    i = 0
    timed_out = True
    while (timed_out == True) and (i <= 5):
        timed_out = False
        response = requests.request('GET', url, headers=headers, params=url_params)

        if 'error' in response.json():
            if response.json()['error']['code'] == 'REQUEST_TIMED_OUT':
                # try again up to 5 times
                timed_out = True
                i += 1

    return response.json()

def obtain_bearer_token(host, path):
    """Given a bearer token, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        str: OAuth bearer token, obtained using client_id and client_secret.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    assert CLIENT_ID, "Please supply your client_id."
    assert CLIENT_SECRET, "Please supply your client_secret."
    data = urlencode({
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': GRANT_TYPE,
    })
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }
    response = requests.request('POST', url, data=data, headers=headers)
    bearer_token = response.json()['access_token']
    return bearer_token

In [77]:
# get authentication and keys
with open('yelp_credentials.json') as data_file:    
    cred = json.load(data_file)


CLIENT_ID = cred['Client ID']
CLIENT_SECRET = cred['Client Secret']
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'

bearer_token = obtain_bearer_token(API_HOST, TOKEN_PATH)

In [78]:
for i, restaurant in restaurants_old_notmerged.iterrows():


    result = request(API_HOST, BUSINESS_PATH, restaurant['google_id'], bearer_token)

    if i == 0:
        restaurants_new_notmerged = pd.DataFrame(pd.io.json.json_normalize(result))
    else:
        df_new = pd.io.json.json_normalize(result)
        restaurants_new_notmerged = restaurants_new_notmerged.append(df_new,ignore_index=True)
    if i % 100 == 0:
        print(i)
        
    # Sometimes API gets stuck after many consecutive call, wait after each call
    time.sleep(0.1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [6]:
'{0}{1}'.format(API_HOST, quote(SEARCH_PATH.encode('utf8')))

'https://api.yelp.com/v3/businesses/search'

In [36]:
pd.io.json.json_normalize(result)

,categories,coordinates.latitude,coordinates.longitude,display_phone,hours,id,image_url,is_claimed,is_closed,location.address1,...,location.state,location.zip_code,name,phone,photos,price,rating,review_count,transactions,url
0,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,(928) 683-6382,"[{'open': [{'is_overnight': False, 'start': '0...",sofias-mexican-food-gila-bend,https://s3-media2.fl.yelpcdn.com/bphoto/WddzkO...,False,False,530 W Pima St,...,AZ,85337,Sofia's Mexican Food,+19286836382,[https://s3-media2.fl.yelpcdn.com/bphoto/Wddzk...,$,4.0,86,[],https://www.yelp.com/biz/sofias-mexican-food-g...


In [34]:
result

{'categories': [{'alias': 'mexican', 'title': 'Mexican'}],
 'coordinates': {'latitude': 32.9456032090449, 'longitude': -112.725908617791},
 'display_phone': '(928) 683-6382',
 'hours': [{'hours_type': 'REGULAR',
   'is_open_now': True,
   'open': [{'day': 0, 'end': '2100', 'is_overnight': False, 'start': '0600'},
    {'day': 1, 'end': '2100', 'is_overnight': False, 'start': '0600'},
    {'day': 2, 'end': '2100', 'is_overnight': False, 'start': '0600'},
    {'day': 3, 'end': '2100', 'is_overnight': False, 'start': '0600'},
    {'day': 4, 'end': '2100', 'is_overnight': False, 'start': '0600'},
    {'day': 5, 'end': '2100', 'is_overnight': False, 'start': '0700'},
    {'day': 6, 'end': '2100', 'is_overnight': False, 'start': '0700'}]}],
 'id': 'sofias-mexican-food-gila-bend',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/WddzkO2_ITK7AWMdhm74JQ/o.jpg',
 'is_claimed': False,
 'is_closed': False,
 'location': {'address1': '530 W Pima St',
  'address2': '',
  'address3': '',
  'city'

In [43]:
i

2

In [79]:
restaurants_new_notmerged.to_pickle('./yelp_dataset_processed/yelp_df_notmerged_new.pkl')

In [80]:
# Repeat the same about the merged restaurants

restaurants_merged = pd.read_pickle('./yelp_dataset_processed/yelp_df.pkl')

In [6]:
restaurants_merged.head()

,categories_new,coordinates,display_phone,distance,id,image_url,is_closed,location,name_new,phone,...,neighborhoods,open,review_count,stars,state,type,zprice,zreview_count,zstar,unique
0,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...","{'latitude': 33.7135958370734, 'longitude': -1...",(623) 825-0355,11.3292,hot-bagels-and-deli-glendale-az,https://s3-media4.fl.yelpcdn.com/bphoto/efZ-8J...,False,"{'address1': '6520 W Happy Valley Rd', 'addres...",Hot Bagels & Deli,+16238250355,...,[],True,14,3.5,AZ,business,0.000000,0.000000,0.000000,1.0
1,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...","{'latitude': 33.3786951, 'longitude': -111.812...",(480) 632-6453,3.4065,jersey-mikes-subs-gilbert-2,https://s3-media1.fl.yelpcdn.com/bphoto/BYIdRj...,False,"{'address1': '891 E Baseline Rd', 'address2': ...",Jersey Mike's Subs,+14806326453,...,[],True,10,3.5,AZ,business,0.000000,-1.122468,-0.083333,0.0
2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 33.61746, 'longitude': -111.92626}",(480) 321-8800,4.31233,sauce-pizza-and-wine-scottsdale-2,https://s3-media2.fl.yelpcdn.com/bphoto/1lTHQh...,False,"{'address1': '14418 N Scottsdale Rd', 'address...",Sauce Pizza and Wine,+14803218800,...,[],True,55,4,AZ,business,-0.071429,0.064650,0.057143,0.0
3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 33.5669898986816, 'longitude': -1...",(602) 870-1111,17.4581,fuddruckers-phoenix-3,https://s3-media4.fl.yelpcdn.com/bphoto/gcmpOL...,False,"{'address1': '8941 North Black Canyon Hwy', 'a...",Fuddruckers,+16028701111,...,[],True,23,4,AZ,business,0.055556,0.488650,0.066667,0.0
4,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...","{'latitude': 33.5818672180176, 'longitude': -1...",(480) 451-1803,1.93633,mcdonalds-scottsdale-5,https://s3-media4.fl.yelpcdn.com/bphoto/ofHjsT...,False,"{'address1': '9251 E Shea Blvd', 'address2': '...",McDonald's,+14804511803,...,[],True,3,2.5,AZ,business,-0.022727,-0.892338,-0.100000,0.0


In [81]:
for i, restaurant in restaurants_merged.iterrows():


    result = request(API_HOST, BUSINESS_PATH, restaurant['id'], bearer_token)

    if i == 0:
        restaurants_new_merged = pd.DataFrame(pd.io.json.json_normalize(result))
    else:
        df_new = pd.io.json.json_normalize(result)
        restaurants_new_merged = restaurants_new_merged.append(df_new,ignore_index=True)
    if i % 100 == 0:
        print(i)
        
    # Sometimes API gets stuck after many consecutive call, wait after each call
    time.sleep(0.1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


In [11]:
restaurants_new_merged.head()

,categories,coordinates.latitude,coordinates.longitude,display_phone,error.code,error.description,hours,id,image_url,is_claimed,...,location.state,location.zip_code,name,phone,photos,price,rating,review_count,transactions,url
0,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",33.713596,-112.200125,(623) 825-0355,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '0...",hot-bagels-and-deli-glendale-az,https://s3-media4.fl.yelpcdn.com/bphoto/efZ-8J...,True,...,AZ,85310,Hot Bagels & Deli,+16238250355,[https://s3-media4.fl.yelpcdn.com/bphoto/efZ-8...,$,3.5,77.0,[],https://www.yelp.com/biz/hot-bagels-and-deli-g...
1,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,(480) 632-6453,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '1...",jersey-mikes-subs-gilbert-2,https://s3-media1.fl.yelpcdn.com/bphoto/BYIdRj...,True,...,AZ,85233,Jersey Mike's Subs,+14806326453,[https://s3-media2.fl.yelpcdn.com/bphoto/eTDf8...,$,3.0,29.0,[pickup],https://www.yelp.com/biz/jersey-mikes-subs-gil...
2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,(480) 321-8800,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '1...",sauce-pizza-and-wine-scottsdale-2,https://s3-media2.fl.yelpcdn.com/bphoto/1lTHQh...,True,...,AZ,85254,Sauce Pizza and Wine,+14803218800,[https://s3-media2.fl.yelpcdn.com/bphoto/1lTHQ...,$$,3.5,163.0,[pickup],https://www.yelp.com/biz/sauce-pizza-and-wine-...
3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,(602) 870-1111,NaN,NaN,NaN,fuddruckers-phoenix-3,https://s3-media4.fl.yelpcdn.com/bphoto/gcmpOL...,True,...,AZ,85021,Fuddruckers,+16028701111,[https://s3-media4.fl.yelpcdn.com/bphoto/gcmpO...,$$,3.5,81.0,[],https://www.yelp.com/biz/fuddruckers-phoenix-3...
4,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",33.581867,-111.881622,(480) 451-1803,NaN,NaN,NaN,mcdonalds-scottsdale-5,https://s3-media4.fl.yelpcdn.com/bphoto/ofHjsT...,False,...,AZ,85258,McDonald's,+14804511803,[https://s3-media4.fl.yelpcdn.com/bphoto/ofHjs...,$,2.5,16.0,[],https://www.yelp.com/biz/mcdonalds-scottsdale-...


In [58]:
restaurants_new_notmerged.head()

,categories,coordinates.latitude,coordinates.longitude,display_phone,error.code,error.description,hours,id,image_url,is_claimed,...,location.state,location.zip_code,name,phone,photos,price,rating,review_count,transactions,url
0,"[{'alias': 'mexican', 'title': 'Mexican'}]",33.480195,-112.081313,(602) 266-1423,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '0...",la-paloma-mexican-food-phoenix,https://s3-media2.fl.yelpcdn.com/bphoto/qUjY7x...,False,...,AZ,85013,La Paloma Mexican Food,+16022661423,[https://s3-media2.fl.yelpcdn.com/bphoto/qUjY7...,$,3.0,18.0,[],https://www.yelp.com/biz/la-paloma-mexican-foo...
1,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,(928) 683-6382,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '0...",sofias-mexican-food-gila-bend,https://s3-media2.fl.yelpcdn.com/bphoto/WddzkO...,False,...,AZ,85337,Sofia's Mexican Food,+19286836382,[https://s3-media2.fl.yelpcdn.com/bphoto/Wddzk...,$,4.0,86.0,[],https://www.yelp.com/biz/sofias-mexican-food-g...
2,"[{'alias': 'delis', 'title': 'Delis'}]",33.423805,-111.735786,(480) 641-4170,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '0...",yee-haws-mesa,https://s3-media2.fl.yelpcdn.com/bphoto/5HfVEs...,False,...,AZ,85205,Yee-haws,+14806414170,[https://s3-media2.fl.yelpcdn.com/bphoto/5HfVE...,$,4.5,3.0,[],https://www.yelp.com/biz/yee-haws-mesa?adjust_...
3,"[{'alias': 'chinese', 'title': 'Chinese'}]",33.568398,-112.100121,(602) 678-4293,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '1...",panda-gourmet-phoenix,https://s3-media3.fl.yelpcdn.com/bphoto/eIsbpn...,True,...,AZ,85021,Panda Gourmet,+16026784293,[https://s3-media3.fl.yelpcdn.com/bphoto/eIsbp...,NaN,3.5,5.0,[],https://www.yelp.com/biz/panda-gourmet-phoenix...
4,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.504062,-111.929431,(480) 945-1928,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '1...",five-guys-scottsdale-2,https://s3-media2.fl.yelpcdn.com/bphoto/xjJkzA...,True,...,AZ,85251,Five Guys,+14809451928,[https://s3-media2.fl.yelpcdn.com/bphoto/xjJkz...,$$,3.5,87.0,[],https://www.yelp.com/biz/five-guys-scottsdale-...


In [75]:
restaurants_new_notmerged.columns

Index(['categories', 'coordinates.latitude', 'coordinates.longitude',
       'display_phone', 'error.code', 'error.description', 'hours', 'id',
       'image_url', 'is_claimed', 'is_closed', 'location.address1',
       'location.address2', 'location.address3', 'location.city',
       'location.country', 'location.cross_streets',
       'location.display_address', 'location.state', 'location.zip_code',
       'name', 'phone', 'photos', 'price', 'rating', 'review_count',
       'transactions', 'url'],
      dtype='object')

In [82]:
restaurants_new_merged.to_pickle('./yelp_dataset_processed/yelp_df_merged_new.pkl')